In [1]:
import itertools
import pandas as pd
from collections import Counter
# import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('movie_bd_v5.csv', parse_dates=['release_date'])

In [3]:
data.sample()

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,DC Comics|Legendary Pictures|Warner Bros.|Syncopy,2008-07-16,8.1,2008


Видно, что в столбцах cast, director, genres, production_companies может быть больше одного наименования

In [4]:
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   imdb_id               1889 non-null   object        
 1   budget                1889 non-null   int64         
 2   revenue               1889 non-null   int64         
 3   original_title        1889 non-null   object        
 4   cast                  1889 non-null   object        
 5   director              1889 non-null   object        
 6   tagline               1889 non-null   object        
 7   overview              1889 non-null   object        
 8   runtime               1889 non-null   int64         
 9   genres                1889 non-null   object        
 10  production_companies  1889 non-null   object        
 11  release_date          1889 non-null   datetime64[ns]
 12  vote_average          1889 non-null   float64       
 13  release_year      

Видно, что пропусков в данных нет. Делать оптимизацию по размеру памяти нет смысла.

# Предобработка

In [5]:
answers = {} # словарь для ответов

In [6]:
# добавим столбец - прибыль фильма
data.insert(3, 'profit', data['revenue'] - data['budget'])

In [7]:
# распакуем столбцы cast, director, genres, production_companies из строк в списки
# обязательно отсортируем записи внутри списка
data['cast'] = data['cast'].str.split(pat='|').map(sorted)
data['director'] = data['director'].str.split(pat='|').map(sorted)
data['genres'] = data['genres'].str.split(pat='|').map(sorted)
data['production_companies'] = data['production_companies'].str.split(pat='|').map(sorted)

In [8]:
# что получилось
data.sample()

,imdb_id,budget,revenue,profit,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
914,tt0268126,19000000,32801173,13801173,Adaptation.,"[Cara Seymour, Chris Cooper, Meryl Streep, Nic...",[Spike Jonze],Charlie Kaufman writes the way he lives... Wit...,A love-lorn script writer grows increasingly d...,114,"[Comedy, Crime, Drama]","[Beverly Detroit, Clinica Estetico, Columbia P...",2002-12-06,7.2,2002


# 1. У какого фильма из списка самый большой бюджет?

In [9]:
data[['budget', 'original_title', 'imdb_id']].sort_values(by='budget', ascending = False).head(1)

,budget,original_title,imdb_id
723,380000000,Pirates of the Caribbean: On Stranger Tides,tt1298650


In [10]:
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'

# 2. Какой из фильмов самый длительный (в минутах)?

In [11]:
data[['runtime', 'original_title', 'imdb_id']].sort_values(by='runtime', ascending = False).head(1)

,runtime,original_title,imdb_id
1157,214,Gods and Generals,tt0279111


In [12]:
answers['2'] = '2. Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [13]:
data[['runtime', 'original_title', 'imdb_id']].sort_values(by='runtime', ascending = True).head(1)

,runtime,original_title,imdb_id
768,63,Winnie the Pooh,tt1449283


In [14]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [15]:
round(data['runtime'].mean())

110

In [16]:
answers['4'] = '2. 110'

# 5. Каково медианное значение длительности фильмов? 

In [17]:
round(data['runtime'].median())

107

In [18]:
answers['5'] = '5. 107'

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [19]:
data[['profit', 'original_title', 'imdb_id']].sort_values(by='profit', ascending = False).head(1)

,profit,original_title,imdb_id
239,2544505847,Avatar,tt0499549


In [20]:
answers['6'] = '5. Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [21]:
data[['profit', 'original_title', 'imdb_id']].sort_values(by='profit', ascending = True).head(1)

,profit,original_title,imdb_id
1245,-165710090,The Lone Ranger,tt1210819


In [22]:
answers['7'] = '5. The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [23]:
data[data['profit'] > 0].shape[0]

1478

In [24]:
answers['8'] = '1. 1478'

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [25]:
data_2008 = data[data['release_year'] == 2008]
data_2008[['profit', 'original_title', 'imdb_id']].sort_values(by='profit', ascending = False).head(1)

,profit,original_title,imdb_id
599,816921825,The Dark Knight,tt0468569


In [26]:
answers['9'] = '4. The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [27]:
data_2012_2014 = data.query('release_year >= 2012 & release_year <= 2014')
data_2012_2014[['profit', 'original_title', 'imdb_id']].sort_values(by='profit', ascending = True).head(1)

,profit,original_title,imdb_id
1245,-165710090,The Lone Ranger,tt1210819


In [28]:
answers['10'] = '5. The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [29]:
# переведём все жанры в один список и посчитаем их
Counter(data['genres'].values.sum()).most_common(1)

[('Drama', 782)]

In [30]:
answers['11'] = '3. Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [31]:
# аналогично только для прибыльных фильмов
Counter(data[data['profit'] > 0]['genres'].values.sum()).most_common(1)

[('Drama', 560)]

In [32]:
answers['12'] = '1. Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [33]:
# столбцы режиссера и сборов взрываем (чтобы кадому режиссеру была отдельная запись),
# группируем по режиссерам суммированием и сортируем
data[['director', 'revenue']].explode('director').groupby(
    'director').sum().sort_values(by='revenue', ascending=False).head(1)

,revenue
director,
Peter Jackson,6490593685


In [34]:
answers['13'] = '5. Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [35]:
# столбцы режиссера и жанров взрываем по жанрам, оставляем только Action,
# столбец режиссеров переводим в один список и берем самых популярных
Counter(data[['director', 'genres']].explode('genres').query('genres == "Action"')['director'].values.sum()).most_common(1)

[('Robert Rodriguez', 9)]

In [36]:
answers['14'] = '3. Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [37]:
# из фильмов 2012 берем столбцы актеров и сборов, взрываем по актерам,
# группируем по актерам суммированием и сортируем
data.query('release_year == 2012')[['cast', 'revenue']].explode(
    'cast').groupby('cast').sum().sort_values('revenue', ascending=False).head(1)

,revenue
cast,
Chris Hemsworth,2027450773


In [38]:
answers['15'] = '3. Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [39]:
# высокобюджетный, это бюджет выше среднего
# возьмем столбец актеров и бюджета и оставим с бюджетом выше среднего
# преобразуем в список и посчитаем
Counter(data['cast'][data['budget'] > data['budget'].mean()].values.sum()).most_common(1)

[('Matt Damon', 18)]

In [40]:
answers['16'] = '3. Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [41]:
# оставим столбцы актеров и жанра, взорвём по актёрам и оставим только Кейджа
# жанры преобразуем в список и посчитаем
Counter(data[['cast', 'genres']].explode('cast').query('cast == "Nicolas Cage"')['genres'].values.sum()).most_common(1)

[('Action', 17)]

In [42]:
answers['17'] = '2. Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [43]:
# оставим столбцы студий, названия и прибыли, взорвём по студиям и оставим только Парамаунт
# остальное отсортируем по прибыли
data[['production_companies', 'original_title', 'profit']].explode('production_companies').query(
    'production_companies == "Paramount Pictures"').sort_values(by='profit').head(1)

,production_companies,original_title,profit
925,Paramount Pictures,K-19: The Widowmaker,-64831034


In [44]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [45]:
# сделаем сводную таблицу год - прибыль по сумме сборов и отсортируем по сборам
pd.pivot_table(data, index='release_year', values='revenue',
               aggfunc='sum').sort_values(by='revenue', ascending=False).head(1)

,revenue
release_year,
2015,25449202382


In [46]:
answers['19'] = '5. 2015'

# 20. Какой самый прибыльный год для студии Warner Bros?

In [47]:
# оставим столбцы студий, года и прибыли, взорвём по студиям
data_wb = data[['production_companies', 'release_year', 'profit']].explode('production_companies')
# оставим только студии, где в названии есть Warner
wb_studios = list(filter(lambda x : 'Warner' in x, set(data['production_companies'].values.sum())))
data_wb = data_wb[data_wb['production_companies'].isin(wb_studios)]
# остаив только год и прибыль, сгруппируем по году и отсортируем по прибыли
data_wb[['release_year', 'profit']].groupby('release_year').sum().sort_values(by='profit', ascending=False).head(1)

,profit
release_year,
2014,2292949646


In [48]:
answers['20'] = '1. 2014'

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [49]:
# сделаем сводную таблицу по месяца (.dt.month) и считаем id фильмов
# сохраним её для следующего задания
month_pivot = pd.pivot_table(data, index=data['release_date'].dt.month, values='imdb_id', aggfunc='count')
month_pivot.sort_values(by='imdb_id', ascending=False).head(1)

,imdb_id
release_date,
9,227


In [50]:
answers['21'] = '4. Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [51]:
summer_months = [6, 7, 8]
# используем предыдущий датафрейм
month_pivot.loc[summer_months].sum()

imdb_id    450
dtype: int64

In [52]:
answers['22'] = '2. 450'

# 23. Какой режиссер выпускает (суммарно по годам) больше всего фильмов зимой? 

In [53]:
winter_months = [12, 1, 2]
# оставим режиссеров с зимними фильмами, перегоним в список и пересчитаем
Counter(data['director'][data['release_date'].dt.month.isin(winter_months)].values.sum()).most_common(1)

[('Peter Jackson', 7)]

In [54]:
answers['23'] = '5. Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [55]:
# посчитаем длину названий, получим индекс самого длинного
# выведем какие студии его сняли
data['production_companies'][data['original_title'].str.len().idxmax()]

['Four By Two Productions', 'Twentieth Century Fox Film Corporation']

In [56]:
answers['24'] = '5. Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [57]:
# оставим столбец студий, добавим столбец длины описаний, взорвём по студиям,
# сгруппируем по студиям по среднему и отсортируем
data[['production_companies']].merge(data['overview'].str.len(), left_index=True, right_index=True).explode(
    'production_companies').groupby('production_companies').mean().sort_values(by='overview', ascending=False).head(1)

,overview
production_companies,
Midnight Picture Show,1000.0


In [58]:
answers['25'] = '3. Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 

In [59]:
# покажем названия фильмов по условию, что оценка фильма больше квантили 0,99 оценок фильмов
data['original_title'][data['vote_average'] > data['vote_average'].quantile(q=0.99)]

9                                          Inside Out
34                                               Room
118                                      Interstellar
119                           Guardians of the Galaxy
125                                The Imitation Game
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
599                                   The Dark Knight
872                                       The Pianist
1081    The Lord of the Rings: The Return of the King
1183                          The Wolf of Wall Street
1191                                 12 Years a Slave
1800                                          Memento
Name: original_title, dtype: object

In [60]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [61]:
# столбец актеров (уже отсортирован) скомбинируем по два, преобразуем в список и посчитаем
Counter(data['cast'].map(lambda x : list(itertools.combinations(x, 2))).values.sum()).most_common(3)

[(('Daniel Radcliffe', 'Emma Watson'), 8),
 (('Daniel Radcliffe', 'Rupert Grint'), 8),
 (('Emma Watson', 'Rupert Grint'), 8)]

In [62]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'


# Submission

In [63]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '5. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [64]:
# и убедиться что ничего не пропустил
len(answers)

27